In [1]:
import os
import tensorflow as tf
from tensorflow.keras import regularizers, layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    BatchNormalization,
    Input,
    concatenate,
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPUs available: {len(gpus)}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPUs found, using CPU")

No GPUs found, using CPU


In [4]:
os.makedirs("output", exist_ok=True)

In [6]:
data = pd.read_csv("../Training_Data/realistic_soldier_data.csv")
data.head()

,Timestamp,Temperature,Moisture,Water_Content,SpO2,Fatigue,Drowsiness,Stress,Heart_Rate,Respiration_Rate,Systolic_BP,Diastolic_BP,Efficiency,Hour_of_Day,Hours_into_Shift
0,2023-03-26 03:30:00,37.147075,65.574965,90.000000,98.524038,0.310956,0.337385,0.242247,84.620125,20.000000,125.0,80.0,0.752684,3.5,3.5
1,2023-03-21 11:30:00,36.446820,68.910684,89.948272,98.864150,0.373187,0.381541,0.364989,79.202808,19.533974,125.0,73.0,0.729088,11.5,3.5
2,2023-01-18 16:00:00,37.515222,35.411185,52.031033,87.013975,0.715033,0.720716,0.688127,115.196388,30.000000,137.0,91.0,0.182040,16.0,0.0
3,2023-04-12 07:30:00,36.035552,69.909713,90.000000,96.553309,0.121165,0.134179,0.156915,81.816142,19.878012,125.0,77.0,0.824440,7.5,7.5
4,2023-01-02 15:30:00,36.968820,35.439972,53.609717,85.188908,0.692070,0.670047,0.684669,118.787102,28.790663,134.0,85.0,0.188814,15.5,7.5


In [7]:
data["Timestamp"] = pd.to_datetime(data["Timestamp"])

In [8]:
# Extract time-based features
data["Day_of_Week"] = data["Timestamp"].dt.dayofweek
data["Is_Weekend"] = (data["Day_of_Week"] >= 5).astype(int)

In [9]:
# Create cyclical time features for better representation
data["Hour_sin"] = np.sin(data["Hour_of_Day"] * (2 * np.pi / 24))
data["Hour_cos"] = np.cos(data["Hour_of_Day"] * (2 * np.pi / 24))
data["Shift_sin"] = np.sin(data["Hours_into_Shift"] * (2 * np.pi / 8))
data["Shift_cos"] = np.cos(data["Hours_into_Shift"] * (2 * np.pi / 8))

In [10]:
# Physiological stress index
data["Physio_Stress_Index"] = (
    ((data["Heart_Rate"] - 60) / 60)
    + ((data["Respiration_Rate"] - 12) / 12)
    + ((data["Systolic_BP"] - 110) / 30)
) / 3

In [11]:
# Hydration index
data["Hydration_Index"] = (data["Water_Content"] / 100) * (data["Moisture"] / 70)

In [12]:
# Define feature sets for different aspects of performance
vital_features = [
    "Temperature",
    "SpO2",
    "Heart_Rate",
    "Respiration_Rate",
    "Systolic_BP",
    "Diastolic_BP",
]
environmental_features = ["Moisture", "Water_Content", "Hydration_Index"]
psychological_features = ["Fatigue", "Drowsiness", "Stress", "Physio_Stress_Index"]
temporal_features = ["Hour_sin", "Hour_cos", "Shift_sin", "Shift_cos", "Is_Weekend"]

In [13]:
# Combine all features
all_features = (
    vital_features + environmental_features + psychological_features + temporal_features
)

In [14]:
all_features

['Temperature',
 'SpO2',
 'Heart_Rate',
 'Respiration_Rate',
 'Systolic_BP',
 'Diastolic_BP',
 'Moisture',
 'Water_Content',
 'Hydration_Index',
 'Fatigue',
 'Drowsiness',
 'Stress',
 'Physio_Stress_Index',
 'Hour_sin',
 'Hour_cos',
 'Shift_sin',
 'Shift_cos',
 'Is_Weekend']

In [15]:
plt.figure(figsize=(12, 10))
selected_features = (
    vital_features[:3]
    + environmental_features[:2]
    + psychological_features[:3]
    + ["Efficiency"]
)
correlation = data[selected_features].corr()
sns.heatmap(correlation, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlations")
plt.tight_layout()
plt.savefig("output/feature_correlations.png")
plt.close()

In [16]:
# Prepare data for modeling
X = data[all_features]
y = data["Efficiency"]

In [18]:
X.head(2)

,Temperature,SpO2,Heart_Rate,Respiration_Rate,Systolic_BP,Diastolic_BP,Moisture,Water_Content,Hydration_Index,Fatigue,Drowsiness,Stress,Physio_Stress_Index,Hour_sin,Hour_cos,Shift_sin,Shift_cos,Is_Weekend
0,37.147075,98.524038,84.620125,20.000000,125.0,80.0,65.574965,90.000000,0.843107,0.310956,0.337385,0.242247,0.525667,0.793353,0.608761,0.382683,-0.92388,1
1,36.446820,98.864150,79.202808,19.533974,125.0,73.0,68.910684,89.948272,0.885485,0.373187,0.381541,0.364989,0.482626,0.130526,-0.991445,0.382683,-0.92388,0


In [19]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [20]:
# Create efficiency bins for stratified sampling
y_bins = pd.cut(y, bins=10, labels=False)

In [21]:
X_train_val, X_test, y_train_val, y_test, bins_train_val, bins_test = train_test_split(
    X_scaled, y, y_bins, test_size=0.15, random_state=42, stratify=y_bins
)

In [22]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val,
    y_train_val,
    test_size=0.15 / 0.85,
    random_state=42,
    stratify=bins_train_val,
)

In [23]:
print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

Training set: 3570 samples
Validation set: 765 samples
Test set: 765 samples


In [24]:
# Build multi-branch model with feature representation learning
def build_enhanced_model(input_shape):
    # Main input
    main_input = Input(shape=(input_shape,), name="main_input")

    # Split input into different feature groups
    # Since we're using a Dense model, we'll use slicing to separate feature groups
    vital_count = len(vital_features)
    env_count = len(environmental_features)
    psych_count = len(psychological_features)
    temp_count = len(temporal_features)

    # Branch 1: Vitals processing
    vitals_input = layers.Lambda(lambda x: x[:, :vital_count])(main_input)
    vitals = Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.001))(
        vitals_input
    )
    vitals = BatchNormalization()(vitals)
    vitals = Dropout(0.2)(vitals)
    vitals = Dense(16, activation="relu")(vitals)

    # Branch 2: Environmental processing
    env_input = layers.Lambda(lambda x: x[:, vital_count : vital_count + env_count])(
        main_input
    )
    env = Dense(16, activation="relu", kernel_regularizer=regularizers.l2(0.001))(
        env_input
    )
    env = BatchNormalization()(env)
    env = Dropout(0.2)(env)
    env = Dense(8, activation="relu")(env)

    # Branch 3: Psychological processing
    psych_input = layers.Lambda(
        lambda x: x[:, vital_count + env_count : vital_count + env_count + psych_count]
    )(main_input)
    psych = Dense(16, activation="relu", kernel_regularizer=regularizers.l2(0.001))(
        psych_input
    )
    psych = BatchNormalization()(psych)
    psych = Dropout(0.3)(psych)
    psych = Dense(8, activation="relu")(psych)

    # Branch 4: Temporal processing
    temp_input = layers.Lambda(lambda x: x[:, -temp_count:])(main_input)
    temp = Dense(16, activation="relu", kernel_regularizer=regularizers.l2(0.001))(
        temp_input
    )
    temp = BatchNormalization()(temp)
    temp = Dropout(0.2)(temp)
    temp = Dense(8, activation="relu")(temp)

    # Branch 5: Direct pathway for residual connections
    direct = Dense(
        32, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)
    )(main_input)
    direct = Dropout(0.2)(direct)

    # Merge all branches
    merged = concatenate([vitals, env, psych, temp, direct])

    # Deep representation learning
    x = Dense(128, activation="relu")(merged)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation="relu")(x)
    x = Dropout(0.2)(x)

    # Output layer
    output = Dense(1, activation="linear", name="output")(x)

    # Create and return model
    model = Model(inputs=main_input, outputs=output)
    return model

In [25]:
model = build_enhanced_model(X_train.shape[1])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ main_input          │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 6)         │          0 │ main_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 3)         │          0 │ main_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 4)         │          0 │ main_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 5)         │          0 │ main_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │        224 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │         64 │ lambda_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 16)        │         80 │ lambda_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 16)        │         96 │ lambda_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 32)        │        128 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16)        │         64 │ dense_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16)        │         64 │ dense_4[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16)        │         64 │ dense_6[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 16)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 16)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 16)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 32)        │        608 │ main_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16)        │        528 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 8)         │        136 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 8)         │        136 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 8)         │        136 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 22,553 (88.10 KB)

 Trainable params: 22,137 (86.47 KB)

 Non-trainable params: 416 (1.62 KB)

In [26]:
# Define callbacks for better training
checkpoint_path = "best_efficiency_model.h5"
checkpoint = ModelCheckpoint(
    checkpoint_path, monitor="val_loss", verbose=1, save_best_only=True, mode="min"
)

In [29]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=20, restore_best_weights=True, verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=7, min_lr=0.00001, verbose=1
)

callbacks = [checkpoint, early_stopping, reduce_lr]

In [30]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="mean_squared_error", metrics=["mae", "mse"])

In [31]:
history = model.fit(
    X_train,
    y_train,
    epochs=150,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1,
)

Epoch 1/150
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1241 - mae: 0.7304 - mse: 0.9691
Epoch 1: val_loss improved from inf to 0.40366, saving model to best_efficiency_model.h5


56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 1.1185 - mae: 0.7281 - mse: 0.9635 - val_loss: 0.4037 - val_mae: 0.4464 - val_mse: 0.2494 - learning_rate: 0.0010
Epoch 2/150
45/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4232 - mae: 0.4022 - mse: 0.2693
Epoch 2: val_loss improved from 0.40366 to 0.36280, saving model to best_efficiency_model.h5


56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4164 - mae: 0.3975 - mse: 0.2626 - val_loss: 0.3628 - val_mae: 0.4072 - val_mse: 0.2102 - learning_rate: 0.0010
Epoch 3/150
43/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3122 - mae: 0.3119 - mse: 0.1600
Epoch 3: val_loss improved from 0.36280 to 0.30949, saving model to best_efficiency_model.h5


56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3084 - mae: 0.3081 - mse: 0.1563 - val_loss: 0.3095 - val_mae: 0.3554 - val_mse: 0.1590 - learning_rate: 0.0010
Epoch 4/150
53/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2552 - mae: 0.2542 - mse: 0.1052
Epoch 4: val_loss improved from 0.30949 to 0.26681, saving model to best_efficiency_model.h5


56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2547 - mae: 0.2536 - mse: 0.1047 - val_loss: 0.2668 - val_mae: 0.3036 - val_mse: 0.1187 - learning_rate: 0.0010
Epoch 5/150
47/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2227 - mae: 0.2133 - mse: 0.0750
Epoch 5: val_loss improved from 0.26681 to 0.22863, saving model to best_efficiency_model.h5


...

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0040 - mae: 0.0446 - mse: 0.0035 - val_loss: 0.0011 - val_mae: 0.0189 - val_mse: 5.5177e-04 - learning_rate: 5.0000e-04
Epoch 121/150
48/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0045 - mae: 0.0479 - mse: 0.0040
Epoch 121: val_loss did not improve from 0.00110
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0045 - mae: 0.0479 - mse: 0.0040 - val_loss: 0.0013 - val_mae: 0.0240 - val_mse: 8.1185e-04 - learning_rate: 5.0000e-04
Epoch 122/150
49/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0044 - mae: 0.0472 - mse: 0.0039
Epoch 122: val_loss did not improve from 0.00110

Epoch 122: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0044 - mae: 0.0471 - mse: 0.0039 - val_loss: 0.0014 - val_mae: 0.0241 - val_mse: 8.3812e-04 - learning_rate: 5.0000e-04
Epoch 123/150
45/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0042 - mae: 0.0462 - mse: 0.0037
Epoch 123: val_loss did not 


Epoch 129: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0040 - mae: 0.0447 - mse: 0.0036 - val_loss: 0.0011 - val_mae: 0.0200 - val_mse: 6.3649e-04 - learning_rate: 2.5000e-04
Epoch 130/150
53/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0040 - mae: 0.0449 - mse: 0.0035
Epoch 130: val_loss improved from 0.00109 to 0.00108, saving model to best_efficiency_model.h5


56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0040 - mae: 0.0449 - mse: 0.0035 - val_loss: 0.0011 - val_mae: 0.0214 - val_mse: 6.3991e-04 - learning_rate: 1.2500e-04
Epoch 131/150
49/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0039 - mae: 0.0449 - mse: 0.0035
Epoch 131: val_loss did not improve from 0.00108
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0039 - mae: 0.0448 - mse: 0.0035 - val_loss: 0.0013 - val_mae: 0.0247 - val_mse: 8.4250e-04 - learning_rate: 1.2500e-04
Epoch 132/150
50/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0041 - mae: 0.0454 - mse: 0.0037
Epoch 132: val_loss did not improve from 0.00108
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0041 - mae: 0.0455 - mse: 0.0037 - val_loss: 0.0011 - val_mae: 0.0212 - val_mse: 6.9737e-04 - learning_rate: 1.2500e-04
Epoch 133/150
49/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0044 - mae: 0.0478 - mse: 0.0040
Epoch 133: val_loss did not improve from 0.00108
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0044 - ma

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0043 - mae: 0.0462 - mse: 0.0039 - val_loss: 0.0010 - val_mae: 0.0202 - val_mse: 6.1005e-04 - learning_rate: 6.2500e-05
Epoch 140/150
55/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0037 - mae: 0.0434 - mse: 0.0033
Epoch 140: val_loss did not improve from 0.00101
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0037 - mae: 0.0434 - mse: 0.0033 - val_loss: 0.0011 - val_mae: 0.0226 - val_mse: 7.3721e-04 - learning_rate: 6.2500e-05
Epoch 141/150
53/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0045 - mae: 0.0491 - mse: 0.0041
Epoch 141: val_loss improved from 0.00101 to 0.00096, saving model to best_efficiency_model.h5


56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0045 - mae: 0.0490 - mse: 0.0041 - val_loss: 9.5755e-04 - val_mae: 0.0192 - val_mse: 5.5876e-04 - learning_rate: 6.2500e-05
Epoch 142/150
37/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0039 - mae: 0.0440 - mse: 0.0035
Epoch 142: val_loss did not improve from 0.00096
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0039 - mae: 0.0443 - mse: 0.0035 - val_loss: 0.0010 - val_mae: 0.0208 - val_mse: 6.5217e-04 - learning_rate: 6.2500e-05
Epoch 143/150
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0038 - mae: 0.0445 - mse: 0.0034
Epoch 143: val_loss did not improve from 0.00096
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0038 - mae: 0.0445 - mse: 0.0034 - val_loss: 0.0011 - val_mae: 0.0212 - val_mse: 6.9000e-04 - learning_rate: 6.2500e-05
Epoch 144/150
26/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0040 - mae: 0.0450 - mse: 0.0036 
Epoch 144: val_loss did not improve from 0.00096
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0040

In [32]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Loss Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history["mae"], label="Train MAE")
plt.plot(history.history["val_mae"], label="Validation MAE")
plt.title("Mean Absolute Error Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("MAE")
plt.legend()

plt.tight_layout()
plt.savefig("output/training_history.png")
plt.close()

In [33]:
test_loss, test_mae, test_mse = model.evaluate(X_test, y_test, verbose=0)
test_rmse = np.sqrt(test_mse)

print(f"Test Loss (MSE): {test_loss:.4f}")
print(f"Test MAE: {test_mae:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

Test Loss (MSE): 0.0009
Test MAE: 0.0187
Test RMSE: 0.0224


In [34]:
y_pred = model.predict(X_test).flatten()

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


In [35]:
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2:.4f}")

R² Score: 0.9915


In [36]:
# Plot actual vs predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--")
plt.xlabel("Actual Efficiency")
plt.ylabel("Predicted Efficiency")
plt.title(f"Predicted vs Actual Efficiency (R² = {r2:.4f})")
plt.grid(True)
plt.savefig("output/prediction_results.png")
plt.close()

In [40]:
# Feature importance analysis using permutation importance
from sklearn.inspection import permutation_importance


In [42]:
def predict_wrapper(X):
    return model.predict(X).flatten()

In [43]:
# The error occurs because permutation_importance expects an estimator with a fit method
# Create a scikit-learn compatible wrapper class for our Keras model
from sklearn.base import BaseEstimator, RegressorMixin

class KerasRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, model):
        self.model = model
        
    def fit(self, X, y):
        # This method is required but we don't need to implement it
        # since our model is already trained
        return self
        
    def predict(self, X):
        return self.model.predict(X).flatten()

# Create a proper scikit-learn compatible estimator
keras_estimator = KerasRegressor(model)

# Now use the estimator with permutation_importance
result = permutation_importance(
    keras_estimator, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
)


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [44]:
feature_importance = pd.DataFrame(
    {"Feature": all_features, "Importance": result.importances_mean}
).sort_values("Importance", ascending=False)

In [45]:
print(feature_importance.head(10))

             Feature  Importance
8    Hydration_Index    0.202957
7      Water_Content    0.152352
1               SpO2    0.061179
2         Heart_Rate    0.025162
11            Stress    0.022970
10        Drowsiness    0.021653
3   Respiration_Rate    0.020898
9            Fatigue    0.020195
6           Moisture    0.009878
4        Systolic_BP    0.005539


In [46]:
plt.figure(figsize=(12, 8))
sns.barplot(x="Importance", y="Feature", data=feature_importance.head(15))
plt.title("Feature Importance (Permutation Method)")
plt.tight_layout()
plt.savefig("output/feature_importance.png")
plt.close()

In [48]:
# Error distribution analysis
errors = y_pred - y_test
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(errors, bins=30, alpha=0.7)
plt.title("Error Distribution")
plt.xlabel("Prediction Error")
plt.ylabel("Frequency")

plt.subplot(1, 2, 2)
plt.scatter(y_pred, errors, alpha=0.5)
plt.axhline(y=0, color="r", linestyle="-")
plt.title("Prediction Error vs Predicted Value")
plt.xlabel("Predicted Value")
plt.ylabel("Error")

plt.tight_layout()
plt.savefig("output/error_analysis.png")
plt.close()


In [49]:
model.save("enhanced_efficiency_model.h5")

In [50]:
with open("efficiency_scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

with open("feature_list.pkl", "wb") as f:
    pickle.dump(all_features, f)

In [51]:
def predict_efficiency(new_data, model_path, scaler_path, feature_list_path):
    """
    Predict efficiency for new soldier data

    Parameters:
    new_data (DataFrame): DataFrame containing required features
    model_path: Path to the saved model
    scaler_path: Path to the saved scaler
    feature_list_path: Path to the saved feature list

    Returns:
    Array of predicted efficiencies
    """
    # Load model and preprocessing components
    model = tf.keras.models.load_model(model_path)

    with open(scaler_path, "rb") as f:
        scaler = pickle.load(f)

    with open(feature_list_path, "rb") as f:
        feature_list = pickle.load(f)

    # Process data
    processed_data = new_data.copy()

    # Process timestamp if present
    if "Timestamp" in processed_data.columns:
        processed_data["Timestamp"] = pd.to_datetime(processed_data["Timestamp"])
        if "Hour_of_Day" not in processed_data.columns:
            processed_data["Hour_of_Day"] = (
                processed_data["Timestamp"].dt.hour
                + processed_data["Timestamp"].dt.minute / 60
            )

        processed_data["Day_of_Week"] = processed_data["Timestamp"].dt.dayofweek
        processed_data["Is_Weekend"] = (processed_data["Day_of_Week"] >= 5).astype(int)

        # Create cyclical features
        processed_data["Hour_sin"] = np.sin(
            processed_data["Hour_of_Day"] * (2 * np.pi / 24)
        )
        processed_data["Hour_cos"] = np.cos(
            processed_data["Hour_of_Day"] * (2 * np.pi / 24)
        )

    # Create shift features if needed
    if "Hours_into_Shift" in processed_data.columns:
        processed_data["Shift_sin"] = np.sin(
            processed_data["Hours_into_Shift"] * (2 * np.pi / 8)
        )
        processed_data["Shift_cos"] = np.cos(
            processed_data["Hours_into_Shift"] * (2 * np.pi / 8)
        )

    # Create derived features
    if all(
        f in processed_data.columns
        for f in ["Heart_Rate", "Respiration_Rate", "Systolic_BP"]
    ):
        processed_data["Physio_Stress_Index"] = (
            ((processed_data["Heart_Rate"] - 60) / 60)
            + ((processed_data["Respiration_Rate"] - 12) / 12)
            + ((processed_data["Systolic_BP"] - 110) / 30)
        ) / 3

    if all(f in processed_data.columns for f in ["Water_Content", "Moisture"]):
        processed_data["Hydration_Index"] = (processed_data["Water_Content"] / 100) * (
            processed_data["Moisture"] / 70
        )

    # Check for missing features
    missing_features = [f for f in feature_list if f not in processed_data.columns]
    if missing_features:
        raise ValueError(f"Missing features in input data: {missing_features}")

    # Select and scale features
    X_new = processed_data[feature_list]
    X_new_scaled = scaler.transform(X_new)

    # Predict
    predictions = model.predict(X_new_scaled).flatten()

    return predictions

In [53]:
sample_data = data.iloc[:5].copy()
sample_predictions = predict_efficiency(
    sample_data,
    "enhanced_efficiency_model.h5",
    "efficiency_scaler.pkl",
    "feature_list.pkl",
)
sample_predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step


array([0.73863685, 0.7238167 , 0.1971134 , 0.80328774, 0.20278291],
      dtype=float32)

In [54]:
for i, (actual, pred) in enumerate(zip(sample_data["Efficiency"], sample_predictions)):
    print(f"Sample {i+1}: Actual Efficiency = {actual:.4f}, Predicted = {pred:.4f}")

Sample 1: Actual Efficiency = 0.7527, Predicted = 0.7386
Sample 2: Actual Efficiency = 0.7291, Predicted = 0.7238
Sample 3: Actual Efficiency = 0.1820, Predicted = 0.1971
Sample 4: Actual Efficiency = 0.8244, Predicted = 0.8033
Sample 5: Actual Efficiency = 0.1888, Predicted = 0.2028
